In [0]:
df_alunos = spark.table("nttdataeducacao.silver.resultados_2024") 
df_escolas = spark.table("nttdataeducacao.silver.escolas_2024")   

# 2. O Join (Cruzamento)
# A chave é: school_id (Alunos) == cod_entid (Escolas)
df_enrich = df_alunos.join(
    df_escolas,
    df_alunos.school_id == df_escolas.cod_entid,
    "left"
)

# 3. Seleção das Colunas Importantes (Para não ficar uma tabela gigante com 200 colunas)
df_final = df_enrich.select(
    # Dados do Aluno
    df_alunos["*"], # Traz tudo do aluno
    
    # Dados da Escola (Enriquecimento)
    df_escolas["nome_entid"].alias("nome_escola"),
    df_escolas["dependencia"].alias("tipo_rede_censo"), # Federal, Estadual...
    df_escolas["localizacao"].alias("zona_escola")      # Urbana/Rural
)

display(df_final.limit(5))

In [0]:
from pyspark.sql.functions import col, when

# 1. REAPROVEITE: Leitura e Join (Isso aqui é igual ao seu código de Merge)
df_resultados = spark.table("nttdataeducacao.gold.agg_ranking_escolas")
df_escolas = spark.table("nttdataeducacao.silver.escolas_2024")

# Join padrão (Inner Join para pegar só quem tem nas duas)
df_join = df_resultados.join(
    df_escolas,
    df_resultados.school_id == df_escolas.cod_entid
)

# 2. NOVO: A Lógica de Comparação
# O ENEM usa números (1,2,3,4) e o Censo usa texto ('Federal', etc).
# Precisamos traduzir o número para texto para poder comparar.

df_check = df_join.withColumn(
    "tipo_enem_traduzido",
    when(col("school_type_id") == 1, "Federal")
    .when(col("school_type_id") == 2, "Estadual")
    .when(col("school_type_id") == 3, "Municipal")
    .when(col("school_type_id") == 4, "Privada")
    .otherwise("Outros")
).select(
    "school_id",
    "school_city",
    col("tipo_enem_traduzido").alias("O_que_o_ENEM_diz"),
    col("dependencia").alias("O_que_o_Censo_diz")
)

# 3. O Filtro da Mentira (Onde eles discordam)
df_divergencias = df_check.filter(
    col("O_que_o_ENEM_diz") != col("O_que_o_Censo_diz")
)

# Mostra o resultado
print(f"Total de escolas com divergência: {df_divergencias.count()}")
display(df_divergencias)

In [0]:
from pyspark.sql.functions import col, avg, round, count, desc

# 1. Ler a tabela Mestra
df_comparacao_final = df_final \
    .filter(col("school_id") != -1) \
    .groupBy(
        "school_id",      # <--- ADICIONADO AQUI PARA CONFERÊNCIA
        "nome_escola", 
        "school_state"
    ) \
    .agg(
        count("candidate_id").alias("qtd_alunos"),
        # Médias cruas (precisão total)
        avg("score_essay").alias("raw_redacao"),
        avg("score_math").alias("raw_matematica"),
        avg("score_languages").alias("raw_linguagens"),
        avg("score_humanities").alias("raw_humanas"),
        avg("score_nature").alias("raw_natureza")
    ) \
    .withColumn(
        "total_preciso", 
        col("raw_redacao") + col("raw_matematica") + 
        col("raw_linguagens") + col("raw_humanas") + col("raw_natureza")
    ) \
    .filter(col("qtd_alunos") >= 10) \
    .orderBy(desc("total_preciso")) \
    .limit(10) \
    .select(
        col("school_id"), # <--- E ADICIONADO AQUI NA VISUALIZAÇÃO
        col("nome_escola"),
        col("school_state"),
        round(col("total_preciso"), 2).alias("Nota_Total_Ranking"),
        # As partes para conferência visual
        round(col("raw_redacao"), 0).alias("Redacao"),
        round(col("raw_matematica"), 0).alias("Matematica")
    )

display(df_comparacao_final)